We import necessairy packages

In [5]:
require 'nn' --

model = nn.Sequential()
model:add(nn.SpatialConvolution(1, 2, 3,1))

print('CRN net\n' .. model:__tostring())

model:replace(function(module)
   if torch.typename(module) == 'nn.SpatialConvolution'
            or (torch.typename(module) == 'nn.View'
            or torch.typename(module) == 'nn.Linear') then
      return nn.Identity()
   else
      return module
   end
end)

print('CRN net\n' .. model:__tostring())


CRN net
nn.Sequential {
  [input -> (1) -> output]
  (1): nn.SpatialConvolution(1 -> 2, 3x1)
}	
CRN net
nn.Sequential {
  [input -> (1) -> output]
  (1): nn.Identity
}	


In [6]:
model = nn.SpatialBatchNormalization(2)
A = torch.randn(5, 2, 3, 4)
C = model:forward(A)  -- C will be of size `b x m x h x w`
print(#C)

 5
 2
 3
 4
[torch.LongStorage of size 4]



### *Global Architecture*
<p>![](https://raw.githubusercontent.com/tonioecto/psc-profondeur/model_version_1/images/general_structure.png)

*Equation to compute input feature*
<p>$owidth  = floor((width  + 2*padW - kW) / dW + 1)$
<p>$oheight = floor((height + 2*padH - kH) / dH + 1)$

### *ConvBlock 1 ---- d1 to d2 of ResNet50*
<p>![](https://raw.githubusercontent.com/tonioecto/psc-profondeur/model_version_1/images/typical_structure_1.png)

In [3]:
function convBlock1(d0, d1, d2)
    local cat = nn.ConcatTable()
    
    local branch1 = nn.Sequential()
    branch1:add(nn.SpatialConvolution(d0, d1, 1, 1))
    -- branch1:add(nn.SpatialBatchNormalization(d1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d1, d1, 3, 3, 1, 1, 1, 1))
    -- branch1:add(nn.SpatialBatchNormalization(d1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d1, d2, 1, 1))
    -- branch1:add(SpatialBatchNormalization(d2))
    local branch2 = nn.Sequential()
    
    branch2:add(nn.Identity())
    
    cat:add(branch1)
    cat:add(branch2)
    
    net:add(cat)
    net:add(nn.CAddTable())
    net:add(nn.ReLU)
end

### *ConvBlock 2 ----- d1 to d2 of ResNet50*
<p>![](https://raw.githubusercontent.com/tonioecto/psc-profondeur/model_version_1/images/typical_structure_2.png)

In [4]:
function convBlock2(s, d0, d1, d2)
    local cat = nn.ConcatTable()
    
    local branch1 = nn.Sequential()
    branch1:add(nn.SpatialConvolution(d0, d1, 1, 1, s, s))
    --branch1:add(nn.SpatialBatchNormalization(d1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d1, d1, 3, 3, 1, 1, 1, 1))
    -- branch1:add(nn.SpatialBatchNormalization(d1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d1, d2, 1, 1, 1, 1))
    -- branch1:add(nn.SpatialBatchNormalization(d2))
    
    local branch2 = nn.Sequential()
    branch2:add(nn.SpatialConvolution(d0, d2, 1, 1, s, s))
    -- branch2:add(nn.SpatialBatchNormalization(d2))
    
    cat:add(branch1)
    cat:add(branch2)
    
    net:add(cat)
    net:add(nn.CAddTable())
    net:add(nn.ReLU())
end

### *up-projection*
<p> ![](https://raw.githubusercontent.com/tonioecto/psc-profondeur/model_version_1/images/up_projection.png)
<p> $owidth  = (width  - 1) * dW - 2*padW + kW + adjW$
<p> $oheight = (height - 1) * dH - 2*padH + kH + adjH$
<p> module = nn.SpatialReflectionPadding(padLeft, padRight, padTop, padBottom)
<p> module = nn.SpatialFullConvolution(nInputPlane, nOutputPlane, kW, kH, [dW], [dH], [padW], [padH], [adjW], [adjH])

In [87]:
-- implement simple version of up-convolution
function upConvolution(net, d1, d2)
    net:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    net:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    net:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    net:add(nn.ReLU())
end

In [ ]:
-- implement simple version of up-projection
function upProjection(net, d1, d2)
    local cat = nn.ConcatTable()
    
    local branch1 = nn.Sequential()
    branch1:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    branch1:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    branch1:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d2, d2, 3, 3, 1, 1, 1, 1))
    
    local branch2 = nn.Sequential()
    branch2:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    branch2:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    branch2:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    
    cat:add(branch1)
    cat:add(branch2)
    net:add(cat)
    net:add(nn.CAddTable())
    
    net:add(nn.ReLU())
end

In [88]:
-- input size 304x228x3
-- first part pre-trained by Facebook
-- https://github.com/facebook/fb.resnet.torch
model_resnet

In [ ]:

-- verify resnet-50 structure
print('CRN net\n' .. model_resnet:__tostring())

In [ ]:
-- Second step : different kinds of up-projection implementations
net = nn.Sequential()
net:add(model_resnet)

d0
net:add(nn.SpatialConvolution(d0, d0, 1, 1, 1, 1))
--net:add(nn.BatchNormalization())

--build up projection blocks
up_projection = nn.Sequential()
upProjection(up_projection, 1024, 512)
upProjection(up_projection, 512, 256)
upProjection(up_projection, 256, 128)
upProjection(up_projection, 128, 64)

net:add(up_projection)

d_final = 64
net:add(nn.SpatialConvolution(d_final, 1, 3, 3, 1, 1, 1, 1))
net:add(nn.ReLU())

In [ ]:
-- loss function B(berHu)
-- https://github.com/Kaixhin/Atari/blob/master/modules/HuberCriterion.lua